In [37]:
from sklearn.base import BaseEstimator, clone
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics      import r2_score

import numpy as np; import pandas as pd
from typing import List, Union

from bitstring import Bits

In [38]:
!pip install dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.1 MB/s eta 0:00:0000:0100:010m
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached fsspec-2023.5.0-py3-none-any.whl (160 kB)
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Using cached toolz-0.12.0-py3-none-any.whl (55 kB)


In [42]:
from dask import dataframe as dd

ddf = dd.read_csv('tmpw1zguov5.csv')

In [163]:
def return_bitstring(x_seq):
    b = ''.join((str(z) for z in x_seq))
    bit = Bits(bin = b)
    return bit.uint

In [164]:
ddf['bits'] = ddf.loc[:, ddf.columns.str.contains('x')].apply(return_bitstring, axis = 1)

/root/barcode/.venv/lib/python3.10/site-packages/dask/dataframe/core.py:5917: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'int64'))

  warnings.warn(meta_warning(meta))


In [166]:
result = ddf.set_index('bits').groupby('bits').y.mean()

In [167]:
result.compute()

bits
3     -2.990150
6    -17.113382
7    -12.455653
11   -17.097339
16   -10.500701
18   -18.207829
22   -18.127595
Name: y, dtype: float64

In [104]:
result = ddf.set_index('bits').groupby('bits').y.mean()

In [105]:
result.compute()

bits
3     -2.990150
6    -17.113382
7    -12.455653
11   -17.097339
16   -10.500701
18   -18.207829
22   -18.127595
Name: y, dtype: float64

In [128]:
ddf.set_index('bits')

,x_1,x_2,x_3,x_4,x_5,y
npartitions=1,,,,,,
3,int64,int64,int64,int64,int64,float64
22,...,...,...,...,...,...


In [121]:
ddf

,x_1,x_2,x_3,x_4,x_5,y,bits
npartitions=1,,,,,,,
,int64,int64,int64,int64,int64,float64,int64
,...,...,...,...,...,...,...


In [135]:
ddf.divisions

(None, None)

In [548]:
from itertools import product, combinations
def L(barcode_length):
    def barcode_to_beta(barcode):
        if isinstance(barcode, list):
            output = [1] + barcode
        else:
            output = [1] + list(barcode)
        N = len(output)
        for i in range(2, N):
            output += [np.prod(x) for x in combinations(barcode, i)]
    #     output += [np.prod(output)]
        return output
    all_sets = list(set(product([0,1], repeat = barcode_length))); all_sets.sort()
    L = np.array([barcode_to_beta(x) for x in all_sets]).T
    return L


In [549]:
L(3)

array([[1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 1, 1, 0, 0, 1, 1],
       [0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [559]:
from dask import dataframe as dd
from typing import List, Union
import scipy
from scipy.sparse import diags, csc_matrix
from itertools import product, combinations
import math



class dask_parameter_generator():
    def __init__(self, filename,  output_feature_name:str = None):
        self.ddf = self.init_process_dataframe(filename, output_feature_name)

    @staticmethod
    def get_L(barcode_length):
        """
        Since L is a mapping from \beta to \mu, L should be in the sparse.csc format so that each column indicates how each \beta is represented with \mu
        """
        def barcode_to_beta(barcode):
            if isinstance(barcode, list):
                output = [1] + barcode
            else:
                output = [1] + list(barcode)
            N = len(output)
            for i in range(2, N):
                output += [np.prod(x) for x in combinations(barcode, i)]
        #     output += [np.prod(output)]
            return output
        all_sets = list(set(product([0,1], repeat = barcode_length))); all_sets.sort()
        linear_function_L = np.array([barcode_to_beta(x) for x in all_sets]).T.astype(np.int8)
        linear_function_L = csc_matrix(linear_function_L)
        return linear_function_L
    

    def compute(self, return_type:str = 'dask'):
        if return_type == 'dask':
            return dd.from_pandas(self.ddf.compute(), chunksize = 10)
        elif return_type == 'pandas':
            return self.ddf.compute()
        elif return_type == 'numpy':
            return self.ddf.compute().to_numpy()
        else:
            raise ValueError("return type can be one of the followings: 'dask','pandas','numpy'")

    def init_process_dataframe(self, filename, output_feature_name):
        ddf = dd.read_csv(filename)
        if output_feature_name:
            input_features = ~ddf.columns.str.contains(output_feature_name)
        else:
            input_features = ddf.columns.str.contains('x')
        # Finding basic information of the full linear model
        self.num_main_var = sum(input_features); self.sample_size = ddf.shape[0].compute()
        self.num_full_var = 2**self.num_main_var
        self.L = self.get_L(self.num_main_var)

        dtype = self.find_integer_type(input_features)
        ddf['barcode'] = ddf.loc[:, input_features].apply(self.return_bitstring, axis = 1, meta=(None, dtype))
        ddf = ddf.loc[:, ~np.concatenate([input_features, [False]])]
        ddf = ddf.set_index('barcode')
        return ddf

    @staticmethod
    def find_integer_type(input_features):
        Sum = sum(input_features)
        if Sum <= 8:
            return 'uint8'
        elif Sum <= 16:
            return 'uint16'
        elif Sum <= 32:
            return 'uint32'
        elif Sum <= 64:
            return 'uint64'
        else:
            return int

    @staticmethod
    def return_bitstring(x_seq):
        b = ''.join((str(z) for z in x_seq))
        bit = Bits(bin = b)
        return bit.uint

    # Groupby Y Means
    @property
    def full_segment_means(self):
        if hasattr(self, '_full_segment_means'):
            pass
        else:
            self._full_segment_means = self.find_groupby_means()
        return self._full_segment_means

    def find_groupby_means(self):
        """
        Finding the groupby means by the bits
        """
        y_means = self.ddf.groupby('barcode').y.mean()
        return y_means.reset_index().set_index('barcode')

    # SSTO, SSE, MSE, FIM
    @property
    def ssto(self):
        if hasattr(self, '_ssto'):
            pass
        else:
            self.gen_scale_estimates()
        return self._ssto
    
    @property
    def sse(self):
        if hasattr(self, '_sse'):
            pass
        else:
            self.gen_scale_estimates()
        return self._sse


    @property
    def MLE_SCALE(self):
        if hasattr(self, '_MLE_SCALE'):
            pass
        else:
            self.gen_scale_estimates()
        return self._MLE_SCALE
    
    @property
    def U_SCALE(self):
        if hasattr(self, '_UNBIASED_SCALE'):
            pass
        else:
            self.gen_scale_estimates()
        return self._UNBIASED_SCALE
    
    @property
    def MLE_VAR(self):
        if hasattr(self, '_mle_var'):
            pass
        else:
            self.gen_scale_estimates()
        return self._mle_var

    @property
    def U_VAR(self):
        if hasattr(self, '_unbiased_var'):
            pass
        else:
            self.gen_scale_estimates()
        return self._unbiased_var


    def gen_scale_estimates(self):
        ssto, sse = self.find_error_statistics()
        sse = sse.compute()
        self._sse = sse
        self._ssto = ssto.compute()
        if self.num_full_var >= self.sample_size:
            self._unbiased_var = None
            self._UNBIASED_SCALE = None
        else:
            self._unbiased_var = sse/(self.sample_size- self.num_full_var)
            self._UNBIASED_SCALE = math.sqrt(self._unbiased_var)
        self._mle_var = sse/(self.sample_size)
        self._MLE_SCALE = math.sqrt(self._mle_var)

    @property
    def y_mean(self):
        if hasattr(self, '_y_mean'):
            pass
        else:
            self._y_mean = self.ddf.y.mean().compute()
        return self._y_mean
    
    def find_error_statistics(self):
        result = self.ddf.merge(self.full_segment_means, how = 'left', left_index = True, right_index = True, suffixes = ['','_pred'])
        sse = result.apply(lambda x: (x[0]- x[1])**2, axis = 1, meta=(None, 'float64')).sum()
        y_mean = result.y.mean().compute()
        ssto = result.y.map(lambda y: (y-y_mean)**2, meta=(None, 'float64')).sum()
        return ssto, sse
    
    # Covariance and FIM
    @property
    def cell_means_covariance(self):
        if hasattr(self, '_cell_means_covariance'):
            pass
        else:
            self.find_FIM()
        return self._cell_means_covariance
    
    @property
    def cell_means_fim(self):
        if hasattr(self, '_cell_means_fim'):
            pass
        else:
            self.find_FIM()
        return self._cell_means_fim

    def find_FIM(self) -> scipy.sparse._dia.dia_matrix:
        raw_counts = pd.Series(self.ddf.groupby('barcode').y.count().compute().sort_index())
        sparse_diagonal_matrix= diags(raw_counts.reindex(list(range(self.num_full_var)), fill_value = 0))/self.MLE_VAR
        self._cell_means_fim = sparse_diagonal_matrix
        self._cell_means_covariance = diags(list(map(lambda x: 1/x if x else float('inf'), sparse_diagonal_matrix.diagonal())))



In [560]:
gen = dask_parameter_generator('tmpw1zguov5.csv')

In [561]:
gen.get_L(3)

<8x8 sparse matrix of type '<class 'numpy.uint8'>'
	with 27 stored elements in Compressed Sparse Column format>

In [530]:
gen.MLE_SCALE

0.5345302889099425

In [531]:
gen.find_FIM().toarray().sum()*gen.MLE_VAR

10.000000000000002

In [507]:
ssto, sse = gen.find_error_statistics()

In [513]:
gen.MLE_SCALE

0.5345302889099425

In [533]:
fim = gen.find_FIM()

In [543]:
diags(list(map(lambda x: 1/x if x else float('inf'), fim.diagonal()))).toarray()

array([[inf,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., inf,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., inf, ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., inf,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., inf,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., inf]])

In [508]:
gen.MLE_SCALE

In [504]:
gen.sample_size

10

In [494]:
sse.compute()

2.857226297621466

In [455]:
gen.ddf.merge(gen.full_segment_means, how = 'left', left_index = True, right_index = True, suffixes = ['','_pred']).compute()

,y,y_pred
barcode,,
3,-2.990150,-2.990150
6,-17.636538,-17.113382
6,-16.590226,-17.113382
7,-11.383768,-12.455653
7,-13.527537,-12.455653
11,-17.097339,-17.097339
16,-10.500701,-10.500701
18,-18.285188,-18.207829
18,-18.130470,-18.207829


In [421]:
gen.full_segment_means

,barcode,y
0,3,-2.990150
1,6,-17.113382
2,7,-12.455653
3,11,-17.097339
4,16,-10.500701
5,18,-18.207829
6,22,-18.127595


In [419]:
gen

In [420]:
gen.ddf.compute()

KeyError: 7

In [400]:
gen.ddf.index

Dask Index Structure:
npartitions=1
3     uint8
22      ...
Name: barcode, dtype: uint8
Dask Name: sort_index-index, 8 graph layers

In [393]:
gen.ddf

,y
npartitions=1,
3,float64
22,...


In [378]:
gen.full_segment_means.loc[3].y

-17.09733937409042

In [367]:
gen.find_FIM().toarray().sum()

10.0

In [357]:
sdiag = diags(pd.Series(gen.ddf.groupby('barcode').y.count().compute().sort_index()).reindex(list(range(2**5)), fill_value = 0))

In [358]:
type(sdiag)

scipy.sparse._dia.dia_matrix

In [355]:
sdiag.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [352]:
from scipy.sparse import diags


diags()


In [301]:
gen.sample_size

10

In [282]:
gen.gen_basic_parameters()

In [286]:
gen.ddf.shape

(Delayed('int-e2d37237-045f-4e0d-b28c-ecfc07f863f7'), 1)

In [276]:
gen.compute('dask')

,y
npartitions=1,
3,float64
22,...


In [259]:
gen.ddf.compute().to_numpy()

array([[ -2.99015048],
       [-17.63653766],
       [-16.59022621],
       [-11.38376786],
       [-13.52753723],
       [-17.09733937],
       [-10.50070062],
       [-18.28518827],
       [-18.13046978],
       [-18.12759475]])

In [258]:
gen.ddf

,y
npartitions=1,
3,float64
22,...


In [212]:
gen.input_featuers

array([ True,  True,  True,  True,  True, False])

In [204]:
gen.ddf.apply(gen.return_bitstring, axis = 1)

ValueError: Metadata inference failed in `apply`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
TypeError("dask_parameter_generator.return_bitstring() missing 1 required positional argument: 'x_seq'")

Traceback:
---------
  File "/root/barcode/.venv/lib/python3.10/site-packages/dask/dataframe/utils.py", line 193, in raise_on_meta_error
    yield
  File "/root/barcode/.venv/lib/python3.10/site-packages/dask/dataframe/core.py", line 6810, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/root/barcode/.venv/lib/python3.10/site-packages/dask/utils.py", line 1105, in __call__
    return getattr(__obj, self.method)(*args, **kwargs)
  File "/root/barcode/.venv/lib/python3.10/site-packages/pandas/core/frame

In [192]:
gen.ddf

,x_1,x_2,x_3,x_4,x_5,y
npartitions=1,,,,,,
,int64,int64,int64,int64,int64,float64
,...,...,...,...,...,...


In [189]:
ddf = dd.read_csv('tmpw1zguov5.csv')
ddf

,x_1,x_2,x_3,x_4,x_5,y
npartitions=1,,,,,,
,int64,int64,int64,int64,int64,float64
,...,...,...,...,...,...


In [186]:
gen.ddf

<module 'dask.dataframe' from '/root/barcode/.venv/lib/python3.10/site-packages/dask/dataframe/__init__.py'>

In [118]:
dask.dataframe

TypeError: 'module' object is not callable

In [159]:
from bitstring import BitArray, Bits

x = Bits(bin = "111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111")
y = Bits(bin = "1111111111111111111111111111111111")

In [160]:
type(x.uint)

int

In [162]:
type(x.uintbe)

int